In [1]:
from bs4 import BeautifulSoup
import requests
import pickle

import re

import pandas as pd
import numpy as np

from functools import reduce
from tqdm.notebook import tqdm_notebook

from multiprocessing.pool import ThreadPool as Pool

import warnings
warnings.filterwarnings('ignore')

Read links

In [2]:
with open('data/osszes_link.pkl', 'rb') as pkl:
    osszes_link = pickle.load(pkl)

In [3]:
len(osszes_link)

80449

### Get data

In [4]:
valtozok = ['Alaptípus ára:', 'Extrákkal növelt ár:', 'Akciós ár:', 'Akció feltételei:', 'Vételár:', 'Vételár EUR:', 'Átvehető:', 'Évjárat:', 'Állapot:', 'Kivitel:', 'Garancia:',
            'Kilométeróra állása:', 'Szállítható szem. száma:', 'Ajtók száma:', 'Szín:', 'Saját tömeg:', 'Teljes tömeg:',
            'Csomagtartó:', 'Klíma fajtája:', 'Üzemanyag:', 'Hengerűrtartalom:', 'Teljesítmény:',
            'Hajtás:', 'Sebességváltó fajtája:', 'Okmányok jellege:', 'Műszaki vizsga érvényes:']

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [6]:
def scrape_car_listing(url):
    
    page = requests.get(url, headers = headers)
    
    if page.ok == False:
        pass
    
    soup = BeautifulSoup(page.text)
    
    adatlap = soup.find('div', id = 'adatlap')
    
    if len(adatlap) == 0:
        pass

    else:        
        
        marka = soup.find('a', href = True, type = 'marka')
        if marka is not None:
            marka = marka.text
            
        modellcsoport = soup.find('a', href = True, type = 'modellcsoport')
        if modellcsoport is not None:
            modellcsoport = modellcsoport.text
            
        modell = soup.find('a', href = True, type = 'modell')
        if modell is not None:
            modell = modell.text
        
        marka_dict = {'variable' : 'Márka', 'value' : marka}
        modellcsoport_dict = {'variable' : 'Modellcsoport', 'value' : modellcsoport}
        modell_dict = {'variable' : 'Modell', 'value' : modell}

        hirdetes_nev = adatlap.find('div', class_ = 'adatlap-cim').select('h1')[0].text.strip()
        
        kiajanlo_adatok = soup.find_all('div', class_ = 'adatlap-kiajanlo-adatok')
        katalogus_adatok = [i for i in kiajanlo_adatok if 'Katalógus adatok' in str(i)]

        if len(katalogus_adatok) == 1:

            katalogus = katalogus_adatok[0]
            kat_nevek = [i.text for i in katalogus.find_all('div', class_ = 'col-xs-15')]
            kat_ertekek = [i.text for i in katalogus.find_all('div', class_ = 'col-xs-13')]

            katalogus_tabla = pd.DataFrame.from_dict(dict(zip(kat_nevek, kat_ertekek)), orient = 'index').reset_index().rename(columns = {'index' : 'variable', 0 : 'value'})

        else:
            katalogus_tabla = pd.DataFrame(columns = ['variable', 'value'])

        felszereltseg = adatlap.find('div', class_ = 'felszereltseg') 
        
        if felszereltseg is not None:
            felszereltseg_tipus = [i.text for i in felszereltseg.find_all('h4')]
            felszereltseg_lista = [felszereltseg_tipus.find_all('li') for felszereltseg_tipus in felszereltseg.find_all('ul', class_ = 'pontos')]
            felszereltseg_lista = [[felszereltseg.text for felszereltseg in felszereltseg_tipus] for felszereltseg_tipus in felszereltseg_lista]
            felszereltseg_dict = dict(zip(felszereltseg_tipus, felszereltseg_lista))        
            felszereltseg_dict = {'variable' : 'Felszereltség', 'value' : felszereltseg_dict}
            
        else:
            felszereltseg_dict = {'variable' : 'Felszereltség', 'value' : np.nan}
        
        url_dict = {'variable' : 'URL', 'value' : url}
        nev_dict = {'variable' : 'Hirdetés név', 'value'  : hirdetes_nev}
        
        hirdetes_adatok = adatlap.find('table', class_ = 'hirdetesadatok')

        if hirdetes_adatok is None:            
            pass

        else:                
            adatok = pd.read_html(str(hirdetes_adatok))[0]            
            adatok.columns = ['variable', 'value']
            adatok = adatok.loc[adatok['variable'].isin(valtozok)]

            adatok = adatok.append(url_dict, ignore_index = True)
            adatok = adatok.append(nev_dict, ignore_index = True)
            adatok = adatok.append(felszereltseg_dict, ignore_index = True)
            adatok = adatok.append(katalogus_tabla, ignore_index = True)
            adatok = adatok.append(marka_dict, ignore_index = True)
            adatok = adatok.append(modellcsoport_dict, ignore_index = True)
            adatok = adatok.append(modell_dict, ignore_index = True)
            
    adatok.to_csv('data/collector/' + str(re.sub('[^0-9a-zA-Z]+', '_', url)) + '.csv', index = False)
            
    return adatok

Try running on 50 listings

In [9]:
%%time

collection = []

pool = Pool(16)

for link in tqdm_notebook(osszes_link[:50], desc = 'Collecting listings data into data frames'):
    pool.apply_async(scrape_car_listing, (link,), callback = collection.append)

pool.close()
pool.join()

Wall time: 9.6 s


50 listings was 10 seconds, so 80500 listings will be 80500 / 50 * 10 / 60 / 60 hours

In [10]:
80500 / 50 * 10 / 60 / 60

4.472222222222222

### Run scraper

and pray to God connection doesnt break for 4-5 hours...

Saving dataframes as csv during running in case of break

In [11]:
len(osszes_link)

80449

In [12]:
%%time

collection = []

pool = Pool(16)

for link in tqdm_notebook(osszes_link, desc = 'Collecting listings data into data frames'):
    pool.apply_async(scrape_car_listing, (link,), callback = collection.append)

pool.close()
pool.join()

Wall time: 5h 9min 17s


In [13]:
len(collection)

76896

In [14]:
with open('data/osszes_hirdetes.pkl', 'wb') as pkl:
    pickle.dump(collection, pkl)

### Merge dataframes

In [27]:
%%time

autok = pd.concat((df.set_index('variable') for df in collection), axis = 1, join = 'outer').T

Wall time: 1min 14s


In [25]:
# %%time

# autok = reduce(lambda a, b: pd.merge(a, b, on = 'variable', how = 'outer'), collection[:100]).T
# autok.reset_index(drop = True, inplace = True)
# autok.columns = autok.iloc[0]
# autok.drop(autok.index[0], inplace = True)

Wall time: 394 ms


In [28]:
autok.shape

(76896, 52)

In [31]:
autok.reset_index(drop = True, inplace = True)

In [34]:
autok.to_csv('data/autok_final.csv.gz', index = False, compression='gzip')

#### Which links did I lose during scraping?

In [32]:
lost_urls = np.setdiff1d(osszes_link, autok['URL'].tolist()).tolist()
len(lost_urls)

3553

These have since been removed